In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd/content/drive/MyDrive/2023 winter project/test

/content/drive/MyDrive/2023 winter project/test


In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))]
    )

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=8, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

cpu is available


In [7]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5) # 합성곱 연산 (입력 채널 수: 3, 출력 채널 수: 6, 필터 크기: 5x5, stride=1(default))
    self.pool1 = nn.MaxPool2d(2,2) # 합성곱 연산 (필터크기 2x2, stride=2)
    self.conv2 = nn.Conv2d(6, 16, 5) # 합성곱 연산 (입력 채널 수: 6, 출력 채널수: 16, 필터 크기: 5x5, stride=1(default))
    self.pool2 = nn.MaxPool2d(2, 2) # 합성곱 연산 (필터크기 2x2, stride=2)
    self.fc1 = nn.Linear(16 * 5 * 5, 120) # 5x5 피쳐맵 16개를 일렬로 피면 16*5*5개의 노드가 생성됨.
    self.fc2 = nn.Linear(120, 10)
    
  def forward(self, x):
    x = self.pool1(F.relu(self.conv1(x))) # conv1 -> ReLU -> pool1
    x = self.pool2(F.relu(self.conv2(x))) # conv2 -> ReLU -> pool2

    x = x.view(-1, 16 * 5 * 5) # 5x5 피쳐맵 16개를 일렬로 만든다.
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))

    return x

net = Net().to(device) # 모델 선언
print(net)
# 피쳐맵은 다음과 같이 바뀌면서 진행된다. 32 -> 28 -> 14 -> 14 -> 5 

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
)


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9)

In [10]:
loss_ = [] # loss 저장용 리스트
n = len(trainloader) # 배치개수

for epoch in range(10): # 10회 반복

  running_loss = 0.0

  for i, data in enumerate(trainloader, 0):
    
    inputs, labels = data[0].to(device), data[1].to(device) # 배치 데이터

    optimizer.zero_grad() # 배치마다 optimizer 초기화

    outputs = net(inputs) # 노드 10개짜리 예측값 산출
    loss = criterion(outputs, labels) # 크로스 엔트로피 손실함수 계산    optimizer.zero_grad() # 배치마다 optimizer 초기화
    loss.backward() # 손실함수 기준 역전파 
    optimizer.step() # 가중치 최적화

    running_loss += loss.item()
                          

    #loss.backward() # 손실함수 기준 역전파 
    optimizer.step() # 가중치 최적화

    running_loss += loss.item()
  
  loss_.append(running_loss / n)
  print('[%d] loss: %.3f' %(epoch + 1, running_loss / len(trainloader)))

[1] loss: 3.658
[2] loss: 2.709
[3] loss: 2.441
[4] loss: 2.262
[5] loss: 2.138
[6] loss: 2.021
[7] loss: 1.930
[8] loss: 1.864
[9] loss: 1.796
[10] loss: 1.739
